#SUMÁRIO DO PROJETO

1. Qual o contexto da problemática? O que queremos resolver?
  - Empresa de Streamming de Música: Ela tem uma abordagem reativa em relação a clientes que não renovam a assinatura oferecendo 3 meses grátis para os que não renovam. Foi entendido que uma abordagem proativa pode ser mais efetiva.

2. Quais DADOS temos para agregar na solução?
  - Temos um dicionario de dados a seguir.

3. Quais INFORMAÇÕES podemos extrair desses DADOS?
  - aqui irei analisar as possiveis variaveis que podem ser criadas (Feature Engineering)

4. Como chegar na solução e quais métodos/tecnologias podem apoiar e quais não podem? (justificação dos usos, questões de negócio, performance, processamento computacional, recursos e implantação em uma hipotética produtização)
  - a

5. Ótica de negócios: Ganhos $$
  - a


para encaixar no sumário:

1- EAD
- hipoteses voltadas ao problema e para a modelagem
- implicações das análises
- usar o EDA para tomada de decisões

2- Dataprep + feature engeneering + feature selection
- criação do publico-alvo, target, análise de volumetrias, missing, outliers, etc.
- construção de novas features / transformação de features
- feature selection adequado, variaveis significativas para o problema

3- Modelagem
- Estrutura do modelo utilizado, funcionamento, hiperparametros, vantagens e fraquezas
- Modelagem e pipe line adequados ao problema? eu entendo isso?
- Compreendo os impactos ao utilizar outras tecnicas de modelagem e pipeline construidos?

4- Validação
- Validação adequada? (Leakage, overfit, underfit, tamanho da amostra)
- Formas de validação exploradas? (cross-validation, train test split, out of sample, out of time, etc)
- Como as alterações do problema impactam na validação escolhida?

5- Definição e avaliação dos resultados
- Métricas de avaliação adequadas? (MAE, RMSE, Recall, AUC, Silhueta, etc)
- Como outras métricas impactariam os resultados do case?
- Como mudanças na estrutura do problema impactariam diferentes métricas?

6- Qualidade do código
- Organização e boas praticas do codigo (nomes intuitivos, comentários, ordenamento intuitivo)
- Codigo padronizado? (classes, funções e variaveis com nomes padronizados, clean code, pep8, etc?)
- Código visa a implementação? (modularização, tempo de processamento, versionamento etc?)

7- Soft Skills
- Sucinto e dentro do tempo indicado
- Clareza nas explicações
- Apresentação estruturada (começo, meio e fim)



#Configuração do ambiente inicial


Instalação do Java

Instalação do Apache Spark

Configuração das variaveis ambiente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
!tar xf spark-3.4.4-bin-hadoop3.tgz
!pip install -q findspark

#Acessando os dados no drive

In [ ]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.4-bin-hadoop3"

# tornar o pyspark "importável"
import findspark
findspark.init()

# iniciar uma sessão local
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('Case_dataMaster_Victor').getOrCreate()

#Importando os dados

In [ ]:
df_members = spark.read.parquet('/content/drive/MyDrive/Colab Notebooks/CASE_DATA_MASTER_2024-25/members.parquet')
df_transactions = spark.read.parquet('/content/drive/MyDrive/Colab Notebooks/CASE_DATA_MASTER_2024-25/transactions.parquet')
df_userlogs = spark.read.parquet('/content/drive/MyDrive/Colab Notebooks/CASE_DATA_MASTER_2024-25/user_logs.parquet')

#Inicio de EDA (Exploratory Data and Analysis)

In [ ]:
print("Formato da transactions:", df_transactions.count(), "X", len(df_transactions.columns))
df_transactions.printSchema()

print("Formato da user_logs:", df_userlogs.count(), "X", len(df_userlogs.columns))
df_userlogs.printSchema()

print("Formato da members: ", df_members.count(), "X", len(df_members.columns))
df_members.printSchema()

Formato da transactions: 20712225 X 10
root
 |-- msno: string (nullable = true)
 |-- payment_method_id: string (nullable = true)
 |-- payment_plan_days: string (nullable = true)
 |-- plan_list_price: string (nullable = true)
 |-- actual_amount_paid: string (nullable = true)
 |-- is_auto_renew: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- membership_expire_date: string (nullable = true)
 |-- is_cancel: string (nullable = true)
 |-- safra: integer (nullable = true)

Formato da user_logs: 26758971 X 9
root
 |-- msno: string (nullable = true)
 |-- safra: integer (nullable = true)
 |-- num_25: double (nullable = true)
 |-- num_50: double (nullable = true)
 |-- num_75: double (nullable = true)
 |-- num_985: double (nullable = true)
 |-- num_100: double (nullable = true)
 |-- num_unq: double (nullable = true)
 |-- total_secs: double (nullable = true)

Formato da members:  63867246 X 8
root
 |-- msno: string (nullable = true)
 |-- safra: string (nullable = true

#Dicionario de dados

1. Transactions:
  - Colunas:
    - 1º 'mnso': id do usuário;
    - 2º 'payment_method_id': Método de pagamento;
    - 3º 'payment_plan_days': tamanho do plano em DIAS;
    - 4º 'plan_list_price': preço do respectivo plano em New Taiwan Dollar (NTD);
    - 5º 'actual_amount_paid': Total pago atual (nao entendi);
    - 6º 'is_auto_renew': Renovação automatica 1 ou 0;
    - 7º 'transaction_date': data da transação no formato %Y %M %D;
    - 8º 'membership_expire_date': quando o plano vence no formato %Y %M %D;
    - 9º 'is_cancel': se nessa transação o cliente cancelou ou não o plano;
    - 10º 'safra': data (nao entendi 100% ao que se refere essa data).

2. User_logs:
  - Colunas:
    - 1º 'msno': id do usuário;
    - 2º 'safra': formato %Y %M %D (nao entendi essa data);
    - 3º 'num_25': quantidade de músicas tocadas menos de 25% da duração dela;
    - 4º 'num_50': quantidade de músicas tocadas entre 25% e 50% da duração dela;
    - 5º 'num_75': quantidade de músicas tocadas entre 50% e 75% da duração dela;
    - 6º 'num_985': quantidade de músicas tocadas entre 75% e 98,5% da duração dela;
    - 7º 'num_100': quantidade de músicas tocadas acima de 98,5%;
    - 8º 'num_unq': quantidade de musicas tocadas apenas 1 vez;
    - 9º 'total_secs': total de segundos tocados.

3. Members:
  - Colunas:
    - 1º 'msno': id do usuário;
    - 2º 'city': cidade do usuário?
    - 3º 'bd': idade? obs: outliers em 2015
    - 4º 'gender': gênero
    - 5º 'registered_via': meio de registro do cliente
    - 6º 'registration_init_time': primeiro regristo do cliente formato %Y %M %D
    - 7º 'safra': (nao entendi essa data)
    - 8º 'is_ativo': se o cliente esta ativo ou não